In [1]:
import logging

import optuna
import pandas as pd
import lightgbm as lgb

from joblib import parallel_backend
from optuna.trial import TrialState
from optuna.study import MaxTrialsCallback

from preprocessing import get_data

logger = logging.getLogger(__name__)


In [2]:
ls -l /data/preprocessed

total 286776
-rw-r--r-- 1 root root  61106933 Dec  1 15:07 test.parquet
-rw-r--r-- 1 root root 172255989 Dec  1 15:06 train.parquet
-rw-r--r-- 1 root root  60291128 Dec  1 15:06 vali.parquet


##### Functions

In [6]:
def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)

    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)

        logger.info(f"Trial {frozen_trial.number} finished with "
                    f"best value: {frozen_trial.value} "
                    f"and parameters: {frozen_trial.params}. ")


def objective_with_pruning_callback(trial, data_dir: str):
    """ Optuna objective

    In this implementation optuna can stop the training if the score is not good enough,
    I guess.
    """

    data_tr, data_va, data_te = get_data(data_dir=data_dir)

    ds_tr = lgb.Dataset(
        data=data_tr['x'],
        label=data_tr['y'],
        group=data_tr['sizes'],
        reference=None)

    ds_va = lgb.Dataset(
        data=data_va['x'],
        label=data_va['y'],
        group=data_va['sizes'],
        reference=ds_tr)

    lgbm_parameters = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'n_estimators': 400,
        'num_leaves': trial.suggest_categorical('num_leaves', choices=[2**7-1, 2**8-1]),
        'max_depth': trial.suggest_categorical('max_depth', choices=[6, 8, 10]),
        # 'learning_rate': trial.suggest_categorical('learning_rate', choices=[.02, .05]),
        'learning_rate': .03,
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-4, .1, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'verbose': 0,
        'early_stopping_rounds': 30,
        'first_metric_only': True,
        'ndcg_eval_at': [5, 10],
        'device_type': 'gpu' # 'cuda'  # 'gpu'
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(
        trial,
        metric='ndcg@5',
        valid_name='val')

    gbm = lgb.train(
        params=lgbm_parameters,
        train_set=ds_tr,
        valid_sets=[ds_tr, ds_va],
        valid_names=['train', 'val'],
        callbacks=[lgb.log_evaluation(period=10, show_stdv=False),
                   pruning_callback])

    return gbm.best_score["val"]["ndcg@5"]


##### Optimization

In [7]:
# storage = 'sqlite:///src/db.sqlite3'
storage = 'sqlite:///db.sqlite3'
study_name = 'study_02'
data_dir = '/data/preprocessed'


study = optuna.create_study(
    storage=storage,
    pruner=optuna.pruners.PercentilePruner(
        percentile=50,
        n_startup_trials=8,
        n_warmup_steps=50,
        interval_steps=50,
        n_min_trials=8),
    study_name=study_name,
    direction='maximize',
    load_if_exists=True)


study.optimize(
    lambda x: objective_with_pruning_callback(x, data_dir=data_dir),
    n_trials=10,
    n_jobs=1,  # 2
    gc_after_trial=True,
    callbacks=[MaxTrialsCallback(20, states=(TrialState.COMPLETE,)),
               logging_callback])

# with parallel_backend('multiprocessing'):
#     study.optimize(
#         lambda x: objective_with_pruning_callback(x, data_dir=data_dir),
#         n_trials=10,
#         n_jobs=2,  # 2
#         gc_after_trial=True,
#         callbacks=[MaxTrialsCallback(20, states=(TrialState.COMPLETE,)),
#                    logging_callback])


[I 2024-12-01 18:31:34,529] Using an existing study with name 'study_02' instead of creating a new one.
/.pyenv/versions/3.12.7/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning gen

[10]	train's ndcg@5: 0.471638	train's ndcg@10: 0.48659	val's ndcg@5: 0.441913	val's ndcg@10: 0.461458
[20]	train's ndcg@5: 0.486553	train's ndcg@10: 0.498632	val's ndcg@5: 0.450113	val's ndcg@10: 0.469265
[30]	train's ndcg@5: 0.496906	train's ndcg@10: 0.508814	val's ndcg@5: 0.457388	val's ndcg@10: 0.47575
[40]	train's ndcg@5: 0.506885	train's ndcg@10: 0.518738	val's ndcg@5: 0.462986	val's ndcg@10: 0.480852
[50]	train's ndcg@5: 0.515399	train's ndcg@10: 0.526845	val's ndcg@5: 0.465866	val's ndcg@10: 0.484086
[60]	train's ndcg@5: 0.524525	train's ndcg@10: 0.534319	val's ndcg@5: 0.470955	val's ndcg@10: 0.488722
[70]	train's ndcg@5: 0.53279	train's ndcg@10: 0.541214	val's ndcg@5: 0.475445	val's ndcg@10: 0.493218
[80]	train's ndcg@5: 0.539944	train's ndcg@10: 0.54709	val's ndcg@5: 0.478328	val's ndcg@10: 0.496273
[90]	train's ndcg@5: 0.546446	train's ndcg@10: 0.552411	val's ndcg@5: 0.481549	val's ndcg@10: 0.497567
[100]	train's ndcg@5: 0.552253	train's ndcg@10: 0.557753	val's ndcg@5: 0.4820

[I 2024-12-01 18:31:54,238] Trial 1 finished with value: 0.493994139361232 and parameters: {'num_leaves': 127, 'max_depth': 10, 'lambda_l2': 0.000668238436220826, 'min_data_in_leaf': 117}. Best is trial 1 with value: 0.493994139361232.


[290]	train's ndcg@5: 0.62739	train's ndcg@10: 0.622417	val's ndcg@5: 0.493948	val's ndcg@10: 0.511651


/.pyenv/versions/3.12.7/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	train's ndcg@5: 0.481931	train's ndcg@10: 0.489692	val's ndcg@5: 0.444698	val's ndcg@10: 0.462231
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[I 2024-12-01 18:33:59,413] Trial 9 pruned. Trial was pruned at iteration 150.
/.pyenv/versions/3.12.7/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	train's ndcg@5: 0.445938	train's ndcg@10: 0.459673	val's ndcg@5: 0.427695	val's ndcg@10: 0.447614
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[I 2024-12-01 18:34:03,479] Trial 10 pruned. Trial was pruned at iteration 50.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [15]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Explored parameters
trials = study.get_trials(deepcopy=False)
trials_params = pd.DataFrame([trial.params for trial in trials])
trials_params['score'] = [tr.value for tr in trials]
trials_params['start'] = [tr.datetime_start for tr in trials]
trials_params['end'] = [tr.datetime_complete for tr in trials]
# print('\n', trials_params.sort_values('score', ascending=False))

trials_params.sort_values('start')

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  1
  Number of complete trials:  9
Best trial:
  Value:  0.49555216636048705
  Params: 
    num_leaves: 127
    max_depth: 10
    lambda_l2: 0.009435638482405298
    min_data_in_leaf: 93


,num_leaves,max_depth,lambda_l2,min_data_in_leaf,score,start,end
0,127,8,0.040527,120,0.494120,2024-12-01 16:35:22.724584,2024-12-01 16:35:40.887550
1,127,10,0.008860,100,0.493299,2024-12-01 16:35:41.018733,2024-12-01 16:36:01.541991
2,255,6,0.007176,87,0.491819,2024-12-01 16:36:01.645824,2024-12-01 16:36:14.318610
3,127,8,0.037228,168,0.490427,2024-12-01 16:36:14.428018,2024-12-01 16:36:30.583659
4,255,10,0.000115,38,0.492040,2024-12-01 16:36:30.681785,2024-12-01 16:36:56.463613
5,255,8,0.015428,75,0.493980,2024-12-01 16:36:56.559844,2024-12-01 16:37:16.796727
6,255,8,0.008198,180,0.495256,2024-12-01 16:37:16.894185,2024-12-01 16:37:37.276880
7,127,10,0.009436,93,0.495552,2024-12-01 16:37:37.395621,2024-12-01 16:37:56.116224
8,127,6,0.029411,130,0.452959,2024-12-01 16:37:56.233800,2024-12-01 16:37:59.777720
9,255,8,0.008827,51,0.495402,2024-12-01 16:37:59.903236,2024-12-01 16:38:21.300145


In [13]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Explored parameters
trials = study.get_trials(deepcopy=False)
trials_params = pd.DataFrame([trial.params for trial in trials])
trials_params['score'] = [tr.value for tr in trials]
trials_params['start'] = [tr.datetime_start for tr in trials]
trials_params['end'] = [tr.datetime_complete for tr in trials]
# print('\n', trials_params.sort_values('score', ascending=False))

trials_params.sort_values('start')

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10
Best trial:
  Value:  0.495538428085108
  Params: 
    num_leaves: 255
    max_depth: 10
    lambda_l2: 0.008806794586103594
    min_data_in_leaf: 145


,num_leaves,max_depth,lambda_l2,min_data_in_leaf,score,start,end
0,255,6,0.008634,159,0.492380,2024-12-01 16:28:11.597136,2024-12-01 16:28:44.027078
1,255,8,0.001557,43,0.495075,2024-12-01 16:28:11.603660,2024-12-01 16:29:07.611089
2,255,6,0.001807,143,0.492488,2024-12-01 16:28:44.166875,2024-12-01 16:29:13.313997
3,127,6,0.000926,54,0.491708,2024-12-01 16:29:07.755837,2024-12-01 16:29:37.735316
4,255,10,0.008788,68,0.491730,2024-12-01 16:29:13.461578,2024-12-01 16:30:10.583932
5,255,10,0.008807,145,0.495538,2024-12-01 16:29:37.867528,2024-12-01 16:30:44.468969
6,255,6,0.000846,140,0.493844,2024-12-01 16:30:10.713612,2024-12-01 16:30:40.327175
7,127,8,0.005636,50,0.494899,2024-12-01 16:30:40.482692,2024-12-01 16:31:15.561847
8,255,8,0.002044,200,0.490710,2024-12-01 16:30:44.605623,2024-12-01 16:31:19.902471
9,255,8,0.000147,85,0.490874,2024-12-01 16:31:15.689693,2024-12-01 16:31:30.422255


In [12]:
trials_params['end'].max() - trials_params['start'].min()

Timedelta('0 days 00:03:18.825119')

In [11]:
import time

In [21]:
data_dir = '/data/preprocessed'

data_tr, data_va, data_te = get_data(data_dir=data_dir)

ds_tr = lgb.Dataset(
    data=data_tr['x'],
    label=data_tr['y'],
    group=data_tr['sizes'],
    reference=None)

ds_va = lgb.Dataset(
    data=data_va['x'],
    label=data_va['y'],
    group=data_va['sizes'],
    reference=ds_tr)

lgbm_parameters = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'n_estimators': 1000,
    'num_leaves': 255,
    'max_depth': 10,
    'learning_rate': .03,
    'lambda_l2': 1e-2,
    'min_data_in_leaf': 100,
    'verbose': -1, # 0,
    # 'early_stopping_rounds': 10,
    'first_metric_only': True,
    'ndcg_eval_at': [5, 10],
    'device_type': 'gpu' # 'cuda'  # 'gpu'
}

# Please recompile with CMake option -DUSE_CUDA=1

In [22]:
ti = time.time()

gbm = lgb.train(
    params=lgbm_parameters,
    train_set=ds_tr,
    valid_sets=[ds_tr, ds_va],
    valid_names=['train', 'val'],
    callbacks=[lgb.log_evaluation(period=50, show_stdv=False)])

tf = time.time()
print(tf-ti)

/.pyenv/versions/3.12.7/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[50]	train's ndcg@5: 0.55639	train's ndcg@10: 0.55857	val's ndcg@5: 0.470421	val's ndcg@10: 0.486792
[100]	train's ndcg@5: 0.599329	train's ndcg@10: 0.597798	val's ndcg@5: 0.483935	val's ndcg@10: 0.500432
[150]	train's ndcg@5: 0.631832	train's ndcg@10: 0.625856	val's ndcg@5: 0.489032	val's ndcg@10: 0.504422
[200]	train's ndcg@5: 0.65957	train's ndcg@10: 0.650142	val's ndcg@5: 0.490309	val's ndcg@10: 0.506669
[250]	train's ndcg@5: 0.680556	train's ndcg@10: 0.668878	val's ndcg@5: 0.491537	val's ndcg@10: 0.508961
[300]	train's ndcg@5: 0.696708	train's ndcg@10: 0.683668	val's ndcg@5: 0.492413	val's ndcg@10: 0.509424
[350]	train's ndcg@5: 0.707206	train's ndcg@10: 0.693705	val's ndcg@5: 0.492517	val's ndcg@10: 0.50997
[400]	train's ndcg@5: 0.715658	train's ndcg@10: 0.700969	val's ndcg@5: 0.492595	val's ndcg@10: 0.510488
[450]	train's ndcg@5: 0.724008	train's ndcg@10: 0.708806	val's ndcg@5: 0.492319	val's ndcg@10: 0.51059
[500]	train's ndcg@5: 0.730536	train's ndcg@10: 0.715631	val's ndcg@5:

In [20]:
ti = time.time()

gbm = lgb.train(
    params=lgbm_parameters,
    train_set=ds_tr,
    valid_sets=[ds_tr, ds_va],
    valid_names=['train', 'val'],
    callbacks=[lgb.log_evaluation(period=50, show_stdv=False)])

tf = time.time()
print(tf-ti)

/.pyenv/versions/3.12.7/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[50]	train's ndcg@5: 0.556882	train's ndcg@10: 0.558167	val's ndcg@5: 0.469417	val's ndcg@10: 0.487227
[100]	train's ndcg@5: 0.600294	train's ndcg@10: 0.597828	val's ndcg@5: 0.48204	val's ndcg@10: 0.499799
[150]	train's ndcg@5: 0.633379	train's ndcg@10: 0.626511	val's ndcg@5: 0.487894	val's ndcg@10: 0.505315
[200]	train's ndcg@5: 0.659464	train's ndcg@10: 0.650152	val's ndcg@5: 0.488252	val's ndcg@10: 0.506283
[250]	train's ndcg@5: 0.679728	train's ndcg@10: 0.66796	val's ndcg@5: 0.491153	val's ndcg@10: 0.508837
[300]	train's ndcg@5: 0.693526	train's ndcg@10: 0.679317	val's ndcg@5: 0.490796	val's ndcg@10: 0.509082
[350]	train's ndcg@5: 0.704661	train's ndcg@10: 0.690146	val's ndcg@5: 0.492733	val's ndcg@10: 0.510931
[400]	train's ndcg@5: 0.712296	train's ndcg@10: 0.697109	val's ndcg@5: 0.493555	val's ndcg@10: 0.510801
[450]	train's ndcg@5: 0.719119	train's ndcg@10: 0.703873	val's ndcg@5: 0.492217	val's ndcg@10: 0.509988
[500]	train's ndcg@5: 0.725946	train's ndcg@10: 0.710506	val's ndcg